In [1]:
# Import necessary libraries:
from intake_esgf import ESGFCatalog
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

In [2]:
# Defines neccessary fucntions:
def Sanderson(skill_scores, d_q):

    # Calculate model weights using the modified Sanderson Approach equation
    weights_raw = [np.exp(skill_score / d_q) for skill_score in skill_scores]
    # Use the built-in sum() function to sum the elements of the list
    weights_sum = sum(weights_raw)
     # Normalize the weights by dividing by the total sum of weights
    weights = [weight / weights_sum for weight in weights_raw]

    return weights


In [3]:
# Extract desired information from ILAMB
df = pd.read_csv("https://www.ilamb.org/CMIP6/historical/scalar_database.csv")
q = df[
    (df.Variable == "Precipitation")
    & (df.ScalarName == "Overall Score")
    & (df.Source == "GPCPv2.3")
    & (df.Region == "global")
    & (df.AnalysisType == "MeanState")
]

print(q)

       Section       Variable    Source          Model     ScalarName  \
6502  Forcings  Precipitation  GPCPv2.3    UKESM1-0-LL  Overall Score   
6632  Forcings  Precipitation  GPCPv2.3      GFDL-ESM4  Overall Score   
6762  Forcings  Precipitation  GPCPv2.3    BCC-CSM2-MR  Overall Score   
6892  Forcings  Precipitation  GPCPv2.3  MPI-ESM1.2-HR  Overall Score   
7022  Forcings  Precipitation  GPCPv2.3     NorESM2-LM  Overall Score   
7152  Forcings  Precipitation  GPCPv2.3    MIROC-ESM2L  Overall Score   
7282  Forcings  Precipitation  GPCPv2.3          CESM2  Overall Score   
7412  Forcings  Precipitation  GPCPv2.3   IPSL-CM6A-LR  Overall Score   
7542  Forcings  Precipitation  GPCPv2.3  ACCESS-ESM1-5  Overall Score   
7672  Forcings  Precipitation  GPCPv2.3        CanESM5  Overall Score   
7802  Forcings  Precipitation  GPCPv2.3      MeanCMIP6  Overall Score   

     AnalysisType  Region ScalarType Units      Data  Weight  
6502    MeanState  global      score     1  0.744698    40.0

In [4]:
# Extract ILAMB skill score from the dictionary above 
source_mean = df[(df.Variable=="Precipitation") & (df.ScalarName=="Overall Score") & (df.Region=="global") & (df.AnalysisType=="MeanState")& (df.Model != "MeanCMIP6")].drop(columns="Weight").groupby('Model').mean(numeric_only=True)
print(source_mean)

# Convert skill scores to numpy array
skill_score_array = np.array(source_mean['Data'])
print(skill_score_array)

                   Data
Model                  
ACCESS-ESM1-5  0.630472
BCC-CSM2-MR    0.659121
CESM2          0.695304
CanESM5        0.642938
GFDL-ESM4      0.687051
IPSL-CM6A-LR   0.645624
MIROC-ESM2L    0.657760
MPI-ESM1.2-HR  0.678806
NorESM2-LM     0.686514
UKESM1-0-LL    0.682414
[0.6304722 0.6591206 0.6953042 0.6429384 0.6870514 0.6456242 0.6577596
 0.6788058 0.6865144 0.6824142]


In [7]:
# Now use Sanderson function to estimate weights based on ILAMB skill scores
weights_Sanderson_10 = Sanderson(skill_score_array, 0.1)
print(weights_Sanderson_10)

weights_Sanderson_25 = Sanderson(skill_score_array, 0.25)
print(weights_Sanderson_25)

[0.06817123158038363, 0.09078614188269264, 0.13036557920834338, 0.07722202015458655, 0.12003875530566553, 0.07932415234175898, 0.08955891272951759, 0.11053792243313895, 0.11939587486853695, 0.11459940949537592]
[0.08623746596747918, 0.09670821218770323, 0.11176883031490605, 0.0906466991340943, 0.10813944192130305, 0.09162578459081697, 0.0961831631607499, 0.10463092135931014, 0.10790740769403148, 0.10615207366960573]
